In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import win32com.client
import getpass








In [20]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ME2L and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "me2l"
session.findById("wnd[0]").sendVKey(0)

# Set the "we101" text in the appropriate field
session.findById("wnd[0]/usr/ctxtSELPA-LOW").text = "we101"

# Set focus and position for the next field
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/btn%_S_BSART_%_APP_%-VALU_PUSH").press()

# Fill in multiple values in a subsequent popup
values = ["zsto", "zspt", "zvor", "zipl", "zatt"]
for i, value in enumerate(values):
    session.findById(f"wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,{i}]").text = value

# Set focus and caret position
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").caretPosition = 4

# Send VKey 8 to confirm the entries
session.findById("wnd[1]").sendVKey(8)

# Set the plant to "0201"
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").text = "0201"
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").caretPosition = 4

# Send VKey 8 to confirm the plant entry
session.findById("wnd[0]").sendVKey(8)

# Execute the next steps
session.findById("wnd[0]").sendVKey(23)
session.findById("wnd[0]/tbar[1]/btn[33]").press()

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(8, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").firstVisibleRow = 3
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "8"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)\\"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "me2l_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 7
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)




In [2]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/me2l_r3.XLSX")
    book.close()
except Exception as e:
    print(e)

LECTURA DE DATAFRAMES

In [2]:

#ME2L POR SEPARADO
me2l_df = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/me2l_r3.xlsx", dtype={'Posición':'str', 'Documento compras':'str'})

In [3]:
me2l_df = me2l_df.drop(0)

In [4]:
me2l_df['Por calcular (cantidad)'] = pd.to_numeric(me2l_df['Por calcular (cantidad)'], errors='coerce')
me2l_df['Por entregar (cantidad)'] = pd.to_numeric(me2l_df['Por entregar (cantidad)'], errors='coerce')


In [5]:
me2l_df = me2l_df[~((me2l_df['Por calcular (cantidad)']==0) & (me2l_df['Por entregar (cantidad)'] == 0))]

In [6]:
me2l_df.shape

(19493, 37)

Estructura de ME2L

In [7]:
base = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Base Planificable/2025-01 Base Planificable/Base Enero OEM-AXS.xlsx", header=1, engine='openpyxl', sheet_name='Hoja1')
base_df_prov = base[['Proveedor', 'Pais (Proveedor)']]
base_df = base[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde','Proveedor', 'Pais (Proveedor)']]

ESTRUCTURA BASE PLANIFICABLE

In [8]:
from pathlib import Path
#BASE PLANIFICABLE
hoy = datetime.datetime.today()
usuario = getpass.getuser()
ruta = f'C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Bases Indicadores en CSV {hoy.year}-{hoy.month:02d}'
ruta_repo = Path(ruta)

In [9]:

base_df = base_df[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde']]

In [10]:
columnas= ['Nro_pieza_fabricante_1',	'Cod_Actual_1']

#columnas= ['Nro_pieza_fabricante_1',	'Cod_Actual_1']
ruta_cod = ruta_repo.joinpath('COD_ACTUAL.csv')

# Leer el archivo CSV en un DataFrame
cod_actual_df = pd.read_csv(ruta_cod)
cod_actual_df = cod_actual_df[columnas]

In [11]:
base_df_ue = pd.merge(base_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")
base_df_ue['Cod_Actual_1'] = base_df_ue['Cod_Actual_1'].fillna(base_df_ue['Material'])
base_df_ue = base_df_ue[['Cod_Actual_1','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']]
base_df_ue = base_df_ue.rename(columns={'Cod_Actual_1':'Material'})
base_df_ue = base_df_ue.drop_duplicates(subset=['Material'])

TRATAMIENTO DE DFS (OC, POSiCION)

In [12]:
print(f'Dimensiones del df1: \nFILAS:{me2l_df.shape[0]}\nCOLUMNAS: {me2l_df.shape[1]}')



Dimensiones del df1: 
FILAS:19493
COLUMNAS: 37


In [13]:
me2l_df['Material'] = me2l_df['Material'].astype('str')
me2l_df['Material'] = me2l_df['Material'].apply(lambda x: x.split(".")[0])

ULTIMO ESLABON A ME2L

In [14]:
me2l_ue = pd.merge(me2l_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")

In [15]:
me2l_ue['Cod_Actual_1'].fillna(me2l_ue['Material'], inplace=True)

In [16]:
me2l_ue = me2l_ue[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material','Cod_Actual_1', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',  'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)'
       ]]


In [17]:
me2l_ue = me2l_ue.rename(columns={'Material':'Material Antiguo'})
me2l_ue = me2l_ue.rename(columns={'Cod_Actual_1':'Material'})


In [18]:
me2l_df['Material'] = me2l_df['Material'].astype('str')
me2l_df['Material'] = me2l_df['Material'].apply(lambda x: x.split(".")[0])

me2l_ue = pd.merge(me2l_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")
me2l_ue['Cod_Actual_1'].fillna(me2l_ue['Material'], inplace=True)

me2l_ue = me2l_ue[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material','Cod_Actual_1', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',  'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)'
       ]]

me2l_ue = me2l_ue.rename(columns={'Material':'Material Antiguo'})
me2l_ue = me2l_ue.rename(columns={'Cod_Actual_1':'Material'})

base_df_prov = base_df_prov.drop_duplicates(subset=['Proveedor'])
me2l_cruce_sector = pd.merge(me2l_ue, base_df_ue[['Material','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']], left_on="Material", right_on="Material", how="left")
#me2l_ue.to_excel("C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO/me2l_sector.xlsx")
me2l_cruce_sector.shape
me2l_cruce_sector = me2l_cruce_sector[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 
       'Fecha de entrega','Hora', 'Fecha entrega estad.',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Texto breve de material', 'TIPO', 'Corresponde']]
me2l_cruce_sector['Cod_Prov'] = me2l_cruce_sector['Nombre del proveedor'].str.split(' ', expand=True)[0]
me2l_cruce_sector = me2l_cruce_sector.merge(base_df_prov,left_on='Cod_Prov', right_on='Proveedor', how='left')
me2l_cruce_sector['Posición'] = me2l_cruce_sector['Posición'].astype('str')
me2l_cruce_sector['AUX'] = me2l_cruce_sector['Documento compras'] + me2l_cruce_sector['Posición']
me2l_cruce_sector['Origen'] = me2l_cruce_sector['Pais (Proveedor)']
me2l_cruce_sector = me2l_cruce_sector[['AUX','Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual','Origen', 'TIPO',  'Corresponde']]
me2l_base_tr = me2l_cruce_sector[me2l_cruce_sector['NomSector_actual'].notna()]
me2l_base_tr['Corresponde'].fillna(0, inplace=True)
me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_31160\2360838543.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].fillna(0, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_31160\2360838543.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)


In [19]:
carpeta_tubo = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Tubo Semanal"
ruta_tubo = os.listdir(carpeta_tubo)[-2]

destino_tubo = carpeta_tubo + '/' +ruta_tubo + '/' + ruta_tubo  +' - '

In [213]:



me2l_base_tr.to_excel(destino_tubo + 'ME2L R3.xlsx')

In [20]:
me2l_cruce_sector = me2l_base_tr

In [21]:
me2l_cruce_sector['AUX'] = me2l_cruce_sector['Documento compras'] + me2l_cruce_sector['Posición']

In [22]:
me2l_cruce_sector = me2l_cruce_sector[['AUX','Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda',
       'Fecha de entrega','Hora', 'Fecha entrega estad.', 
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual','Origen', 'TIPO',  'Corresponde']]

In [23]:
me2l_base_tr = me2l_cruce_sector[me2l_cruce_sector['NomSector_actual'].notna()]

In [24]:
bases_oc = me2l_base_tr.groupby(['Documento compras'])['Posición'].count().sort_values(ascending=False).reset_index()

In [25]:
bases_oc.shape

(811, 2)

In [40]:


# Calculate the number of rows in each part
num_rows_per_part = len(bases_oc) // 3  # Integer division to get an equal split

# Split the DataFrame into five parts
df_parts = np.array_split(bases_oc, 3)


# Alternatively, you can access the parts using indexing (e.g., df_parts[0], df_parts[1], etc.)

# Each part will have a similar number of rows


c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [41]:
df_parts[0]['Documento compras'].to_clipboard(header=False, index=False)

In [42]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ZMM_MONITOR_ORDEN_CL and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "ZMM_MONITOR_ORDEN_CL"
session.findById("wnd[0]").sendVKey(0)

# Select the specific tab
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC").select()

# Set focus and position for the required field
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/btn%_SO_OC_%_APP_%-VALU_PUSH").press()

# Interact with the buttons in the new window
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Send VKey 8 to confirm the entries
session.findById("wnd[0]").sendVKey(8)

# Press the toolbar button
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarButton("&MB_VARIANT")

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(3, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "3"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()

# Export the data
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").selectContextMenuItem("&XXL")
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = ""
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "monitor_1_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]").sendVKey(4)

# Set the directory path in the new window
session.findById("wnd[2]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[2]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[2]/usr/ctxtDY_PATH").caretPosition = 131
session.findById("wnd[2]/tbar[0]/btn[11]").press()

# Confirm the export
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)


In [43]:
df_parts[1]['Documento compras'].to_clipboard(header=False, index=False)

In [44]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ZMM_MONITOR_ORDEN_CL and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "ZMM_MONITOR_ORDEN_CL"
session.findById("wnd[0]").sendVKey(0)

# Select the specific tab
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC").select()

# Set focus and position for the required field
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/btn%_SO_OC_%_APP_%-VALU_PUSH").press()

# Interact with the buttons in the new window
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Send VKey 8 to confirm the entries
session.findById("wnd[0]").sendVKey(8)

# Press the toolbar button
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarButton("&MB_VARIANT")

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(3, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "3"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()

# Export the data
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").selectContextMenuItem("&XXL")
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = ""
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "monitor_2_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]").sendVKey(4)

# Set the directory path in the new window
session.findById("wnd[2]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[2]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[2]/usr/ctxtDY_PATH").caretPosition = 131
session.findById("wnd[2]/tbar[0]/btn[11]").press()

# Confirm the export
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)


In [32]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/monitor_2_r3.XLSX")
    book.close()
except Exception as e:
    print(e)

In [45]:
df_parts[2]['Documento compras'].to_clipboard(header=False, index=False)

In [46]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ZMM_MONITOR_ORDEN_CL and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "ZMM_MONITOR_ORDEN_CL"
session.findById("wnd[0]").sendVKey(0)

# Select the specific tab
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC").select()

# Set focus and position for the required field
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/btn%_SO_OC_%_APP_%-VALU_PUSH").press()

# Interact with the buttons in the new window
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Send VKey 8 to confirm the entries
session.findById("wnd[0]").sendVKey(8)

# Press the toolbar button
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarButton("&MB_VARIANT")

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(3, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "3"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()

# Export the data
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").selectContextMenuItem("&XXL")
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = ""
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "monitor_3_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]").sendVKey(4)

# Set the directory path in the new window
session.findById("wnd[2]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[2]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[2]/usr/ctxtDY_PATH").caretPosition = 131
session.findById("wnd[2]/tbar[0]/btn[11]").press()

# Confirm the export
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)


In [36]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/monitor_3_r3.XLSX")
    book.close()
except Exception as e:
    print(e)

In [47]:
bases_oc['Documento compras'].to_clipboard(header=False, index=False)

In [48]:
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session = connection.Children(0)
#now = datetime.now()

#session.findById("wnd[0]").maximize
#session.findById("wnd[0]").maximize
session.findById("wnd[0]/tbar[0]/okcd").text = "vl06if"
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").text = ""
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").text = ""
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").setFocus()
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/btn%_IT_EBELN_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]").sendVKey(8)
session.findById("wnd[0]/tbar[1]/btn[18]").press()
session.findById("wnd[0]").sendVKey(33)
session.findById("wnd[1]/usr/lbl[1,7]").setFocus()
session.findById("wnd[1]/usr/lbl[1,7]").caretPosition = 4
session.findById("wnd[1]").sendVKey(2)
session.findById("wnd[0]").sendVKey(43)
session.findById("wnd[1]/usr/cmbG_LISTBOX").setFocus()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[1]/tbar[0]/btn[11]").press()
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)




In [39]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/export.XLSX")
    book.close()
except Exception as e:
    print(e)

In [26]:
lista = os.listdir("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)")

In [42]:
dfs = []
print('Archivos usados (Monitor)')
for i in lista:
    if ('monitor_' in i):
        df = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)" + '/' +i, dtype={'N° MIRO':'str'},index_col=None)
        print(i)
        dfs.append(df)


Archivos usados (Monitor)
monitor_1_r3.XLSX
monitor_2_r3.XLSX
monitor_3_r3.XLSX


In [28]:
df_ee = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/export.XLSX" ,dtype={'Documento compras':'str','Posición modelo':'str'})

In [29]:
print(f'Dimensiones del df1\n: FILAS:{df_ee.shape[0]}\nCOLUMNAS: {df_ee.shape[1]}')

Dimensiones del df1
: FILAS:54729
COLUMNAS: 13


In [43]:
monitor_df = pd.concat(dfs,axis=0, ignore_index=True)

In [44]:
monitor_df.columns

Index(['Prefijo OC', 'Nro. OC SAP', 'Clase Doc. OC', 'Posición OC SAP',
       'Ind. Borrado OC', 'Fecha Creación OC', 'Clase Doc. SP', 'Nro. SOLPED',
       'Posición SOLPED', 'Ind. Borrado SP',
       ...
       'Días del pedido', 'Días en BO', 'Días Pend. Emb.', 'Pedido Cerrado',
       'Material Cotizado', 'Cód. Mat de prov en PO', 'Cantidad en BO',
       'Pet.Ofert.Creado por', 'OC Creado por', 'Marcar para "Cerrar"'],
      dtype='object', length=106)

In [45]:
monitor_df[(monitor_df['Nro. OC SAP']==4200007086) & (monitor_df['Posición OC SAP']==710)]['N° MIRO']

7597     5113523765
7598     5113523765
43186    5113513373
43187    5113513373
Name: N° MIRO, dtype: object

In [56]:
monitor_df.to_excel(destino_tubo + "Monitor R3.xlsx")


In [57]:
df_ee.to_excel(destino_tubo + "Entrega Entrante R3.xlsx")

In [58]:
#monitor_df = monitor_df[['Nro. OC SAP','Clase Doc. OC','Posición OC SAP','Ind. Borrado OC','Clase Doc. SP','Nro. SOLPED','Posición SOLPED','Ind. Borrado SP','Cant. UN. OC','N° MIRO','Posición Factura',
#'Cant. Factura','Fe. Confirmación','Fe. Compromiso','Fe. ATA','TF de ATA','Cantidad en BO','Ind. Borrado PO']]

In [46]:
monitor_df['Nro. OC SAP'] = monitor_df['Nro. OC SAP'].astype('str')
monitor_df['Posición OC SAP'] = monitor_df['Posición OC SAP'].astype('str')


In [47]:
monitor_df['AUX'] = monitor_df['Nro. OC SAP'] + monitor_df['Posición OC SAP']

In [48]:
monitor_df['N° MIRO'] = monitor_df['N° MIRO'].astype('str')

In [49]:
monitor_df['Posición Factura'] = monitor_df['Posición Factura'].astype('str')

In [50]:
monitor_df['AUX_FACTURA'] = monitor_df['N° MIRO'] + monitor_df['Posición Factura']

In [52]:
monitor_df['AUX_FACTURA']

0                 nan0
1                 nan0
2                 nan0
3                 nan0
4                 nan0
             ...      
66211     511330108534
66212     511330108580
66213    5113301085127
66214    5113301085277
66215    5113301085143
Name: AUX_FACTURA, Length: 66216, dtype: object

In [35]:
#revision = monitor_df.groupby('AUX')['Fe. ATA'].nunique().sort_values(ascending=False).reset_index()

In [53]:
tabla_1 = monitor_df[monitor_df['Ind. Borrado OC'].isna() &
    monitor_df['Ind. Borrado SP'].isna() &
    monitor_df['Ind. Borrado PO'].isna()].groupby(['AUX', 'AUX_FACTURA']).agg({'Cant. UN. OC':'mean', 'Cant. Factura':'max','Cantidad en BO':'mean'})

In [54]:
tabla_final = tabla_1.groupby(['AUX']).agg({'Cant. UN. OC':'mean','Cant. Factura':'sum', 'Cantidad en BO':'mean'}).reset_index()

In [55]:
tabla_final[tabla_final['AUX']=='420000706950']

,AUX,Cant. UN. OC,Cant. Factura,Cantidad en BO
34645,420000706950,466.0,424,-182.0


In [56]:
print(f'Dimension de tabla final: {tabla_final.shape}')
print('-'*40)
print(f'Total Campo OC: {tabla_final['Cant. UN. OC'].sum()}')
print(f'Total Campo Factura: {tabla_final['Cant. Factura'].sum()}')
print(f'Total Campo BO: {tabla_final['Cantidad en BO'].sum()}')

Dimension de tabla final: (48300, 4)
----------------------------------------
Total Campo OC: 935537.0
Total Campo Factura: 724840
Total Campo BO: 197707.33333333334


In [57]:
df_ee = df_ee[['Documento compras','Posición modelo', 'Cantidad entrega', 'Status mov.mcía.']]

In [58]:
df_ee['Documento compras'] = df_ee['Documento compras'].astype('str')
df_ee['Posición modelo'] = df_ee['Posición modelo'].astype('str')

In [59]:
df_ee.dtypes

Documento compras    object
Posición modelo      object
Cantidad entrega      int64
Status mov.mcía.     object
dtype: object

In [60]:
df_ee['AUX'] = df_ee['Documento compras'] + df_ee['Posición modelo']
df_ee.drop(columns=['Documento compras','Posición modelo'])

,Cantidad entrega,Status mov.mcía.,AUX
0,2,C,420000413360
1,1,C,420000413370
2,1,C,4200004133130
3,2,C,4200004133190
4,1,C,4200004133260
...,...,...,...
54724,100,C,420000723110
54725,2,C,420000723120
54726,5,C,420000723140
54727,32,NaN,nannan


In [61]:
table_ee = df_ee[df_ee['Status mov.mcía.']=='C'].groupby(['AUX'])['Cantidad entrega'].sum().reset_index()

In [62]:
me2l_base_tr.shape

(19191, 42)

In [63]:
me2l_15 = me2l_base_tr

In [64]:
monitor_df.drop_duplicates(subset=['AUX'], inplace=True)

In [65]:
me2l_15 = me2l_base_tr

In [66]:
me2l_15.dtypes

AUX                                            object
Documento compras                              object
Posición                                       object
Reparto                                       float64
Cl.documento compras                           object
Tipo doc.compras                               object
Grupo de compras                               object
Historial pedido/Docu.orden entrega           float64
Fecha documento                        datetime64[ns]
Material Antiguo                               object
Material                                       object
Texto breve                                    object
Grupo de artículos                            float64
Indicador de borrado                          float64
Tipo de posición                              float64
Tipo de imputación                            float64
Centro                                        float64
Almacén                                       float64
Cantidad de reparto         

In [68]:
me2l_15 = me2l_15.merge(monitor_df[['AUX','Fe. ATA']], left_on='AUX', right_on='AUX', how = 'left')

In [69]:
me2l_15['Fecha Est. Fact'] = np.where(me2l_15['Fe. ATA'].notnull(), me2l_15['Fe. ATA'] + pd.Timedelta(days=15), me2l_15['Fecha de entrega'])

In [70]:
me2l_15 = me2l_15.merge(tabla_final[['AUX', 'Cant. Factura']], left_on = 'AUX', right_on='AUX',how='left')

In [71]:
me2l_15['Cant. Factura'] = me2l_15['Cant. Factura'] * me2l_15['Corresponde']

In [72]:
me2l_15= me2l_15.merge(table_ee, left_on='AUX', right_on='AUX', how='left')

In [73]:
me2l_15['Cantidad entrega'] = me2l_15['Cantidad entrega']*me2l_15['Corresponde']

In [74]:
me2l_15['Qty por Entregar'] = me2l_15['Por entregar (cantidad)']*me2l_15['Corresponde']

In [75]:
#cambiar esto por la fecha del dia actual

hoy = datetime.datetime.today()

In [76]:
table_ee[table_ee['AUX']=='4200007086710']

,AUX,Cantidad entrega


In [77]:
me2l_15['Cantidad entrega'].fillna(0, inplace=True)

In [78]:


# Suponiendo que 'me2l_15' es tu DataFrame y 'AW4', 'BJ4' y 'AT4' son columnas en tu DataFrame
result = me2l_15['Cant. Factura'] - me2l_15['Cantidad entrega']
result = np.where(np.isnan(result), me2l_15['Qty por Entregar'], result)

me2l_15['Qty Fact Corr'] = np.where(result<0, 0, result)

In [79]:
me2l_15[['AUX','Qty Fact Corr']].to_excel('verif_x_entr.xlsx')

In [80]:
hoy = pd.to_datetime(hoy)

In [81]:
me2l_15[ 'Fecha Est. Fact'].max()

Timestamp('2025-06-14 00:00:00')

In [82]:
conditions = [
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] >= hoy),
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] < hoy)
]

choices = ['Facturado No Vencido', 'Facturado Vencido']

me2l_15['Status TR Fact'] = np.select(conditions, choices, default='No Facturado')

In [83]:
from datetime import timedelta, datetime

hoy_31 = hoy  + timedelta(days=31)

In [84]:
hoy_31 = pd.to_datetime(hoy_31)

In [85]:
condition = (me2l_15['Fecha de entrega'] > hoy) & (me2l_15['Fecha de entrega'] < hoy_31)

# Apply the condition using boolean indexing and assign 'SI' or 'NO' accordingly
me2l_15['Fecha BO Mes Actual'] = np.where(condition, 'SI', 'NO')

In [86]:
condition= (me2l_15['Fecha de entrega']< hoy)

me2l_15['TR Vencido'] = np.where(condition, 'SI','NO')

In [87]:
conditions = [
    (me2l_15['Fecha BO Mes Actual'] == "NO") & (me2l_15['TR Vencido'] == "NO"),
    (me2l_15['TR Vencido'] == "SI"),
]

# Define choices
choices = [
    "Fecha Teórica",
    "TR Vencido",
]

# Apply np.select
me2l_15['Statur TR No FAct'] = np.select(conditions, choices, default="Replanificar +45")

In [88]:
#Qty BO
me2l_15['Cant. BO'] = me2l_15['Cant. Factura'].fillna(0, inplace=True)

In [89]:
#Cant Factura : Cant Fact CORR
#QTY BO: CHECK

In [90]:
condicion = (me2l_15['Fecha Est. Fact'] < hoy)

me2l_15['Atraso'] = np.where(condicion, 'SI','NO')

In [91]:
condicion = (me2l_15['Atraso'] == 'SI')

me2l_15['Fecha Facturacion Final'] = np.where(condicion, hoy_31, me2l_15['Fecha Est. Fact'])



In [92]:
me2l_15['Fecha Facturacion Final'] = pd.to_datetime(me2l_15['Fecha Facturacion Final'])

In [93]:
me2l_15 = me2l_15.merge(monitor_df[['AUX', 'Vía (Texto)']], left_on='AUX', right_on='AUX', how='left')

In [94]:
me2l_15['LT Objetivo'] = np.where(me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"]), 45,
                    np.where(me2l_15['Vía (Texto)'] == "Aéreo", 15,
                        np.where(me2l_15['Vía (Texto)'] == "Courier", 7, 0)))

In [95]:
me2l_15 = me2l_15.merge(monitor_df[['AUX','Fe. Confirmación']], left_on='AUX',right_on='AUX', how='left')

In [96]:
me2l_15['Fecha OC'] = np.where((me2l_15['Fe. Confirmación'] == 0) | (me2l_15['Fe. Confirmación'].isna()), me2l_15['Fecha documento'], me2l_15['Fe. Confirmación'])

In [97]:
import datetime
today = hoy

In [98]:
today = pd.to_datetime(today)

In [99]:


# Calculate the difference
me2l_15['Dias LT BO'] = (today - me2l_15['Fecha OC']).dt.days - me2l_15['LT Objetivo']

# Handling potential errors, setting them to 0
me2l_15['Dias LT BO'] = me2l_15['Dias LT BO'].fillna(0)

In [100]:
import pandas as pd
import numpy as np

# Define conditions and choices
conditions = [
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(1, 30)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(31, 60)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(61, 90)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'] > 90),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(1, 15)),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(16, 30)),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(31, 45)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(1, 7)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(8, 15)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(16, 21)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'] > 21),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'] > 45)
]

choices = [
    "De 1 a 30 días",
    "De 31 a 60 días",
    "De 61 a 90 días",
    "De 91 días a más",
    "De 1 a 15 días",
    "De 16 a 30 días",
    "De 31 a 45 días",
    "De 1 a 7 días",
    "De 8 a 15 días",
    "De 16 a 21 días",
    "De 21 días a más",
    "De 45 días a más"
]

# Assign the values based on conditions
me2l_15['Categoría LT'] = np.select(conditions, choices, default="Dentro de LT")



In [101]:
import pandas as pd
import numpy as np

# Assuming CE4 is a column in the DataFrame and BV1 is a variable
BV1 = 45  # Assuming the value of $BV$1 is 45

# Define conditions and choices
conditions = [
    me2l_15['Statur TR No FAct'] == "Replanificar +45",
    me2l_15['Statur TR No FAct'] == "Fecha Teórica",
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & ((me2l_15['Categoría LT'] == 0) | (me2l_15['Categoría LT'] == "Dentro de LT")),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 31 a 60 días"),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 61 a 90 días")
]

choices = [
    hoy + timedelta(days =45),
    me2l_15['Fecha Facturacion Final'],
    hoy + timedelta(days =30),
    hoy + timedelta(days =60),
    hoy + timedelta(days =90)
]

# Assign the values based on conditions
me2l_15['Fecha No Fact'] = np.select(conditions, choices, default=hoy + timedelta(days =120))


In [102]:
me2l_15['Fecha No Fact'] = pd.to_datetime(me2l_15['Fecha No Fact'])

In [281]:
me2l_15= me2l_15.merge(tabla_final[['AUX','Cantidad en BO']], left_on='AUX',right_on='AUX', how='left')

AQUI SE DEJA DE TOMAR MONITOR Y SE TOMA ME2L (POR CALCULAR (CANTIDAD))

In [105]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Fecha de entrega',
       'Hora', 'Fecha entrega estad.', 'Cantidad entrada',
       'Cantidad de salida', 'Cantidad entregada', 'Solicitud de pedido',
       'Pos.solicitud pedido', 'Indicador creación', 'Cantidad de posiciones',
       'Nombre del proveedor', 'Por entregar (cantidad)',
       'Por entregar (valor)', 'Por calcular (cantidad)',
       'Por calcular (valor)', 'NomSector_actual', 'Origen', 'TIPO',
       'Corresponde', 'Fe. ATA', 'Fecha Est. Fact', 'Cant. Factura',
       'Cantidad entrega', 'Qty por Entreg

In [106]:
me2l_15['Cantidad en BO'] = me2l_15['Por calcular (cantidad)']*me2l_15['Corresponde']

In [107]:
me2l_15['Cantidad en BO'].fillna(0, inplace=True)

In [108]:
import numpy as np

# Crear una condición basada en la columna 'TIPO'
condicion = (me2l_15['TIPO'] == 'OEM') & (me2l_15['Por calcular (cantidad)'] == 0)

# Actualizar la columna 'Cantidad en BO' basada en la condición
me2l_15['Cantidad en BO'] = np.where(condicion, 0, me2l_15['Cantidad en BO'])



In [109]:
me2l_15['Por calcular (cantidad)'][me2l_15['AUX']=='420000494760']


Series([], Name: Por calcular (cantidad), dtype: float64)

In [110]:
me2l_15['TIPO'].value_counts()

TIPO
OEM    18937
AXS      254
Name: count, dtype: int64

In [111]:
me2l_15['Cantidad en BO'].sum()

191360.0

In [112]:
me2l_15['Corresponde'].value_counts()

Corresponde
1.0     19188
4.0         2
19.0        1
Name: count, dtype: int64

In [289]:
me2l_15.to_excel(destino_tubo + "Base Transito para Analisis.xlsx")

In [113]:
side_a = me2l_15[['AUX','Status TR Fact',   'Material','Texto breve','Qty Fact Corr','Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha Facturacion Final','Nombre del proveedor','Grupo de compras']]
side_b = me2l_15[['AUX','Statur TR No FAct','Material','Texto breve','Cantidad en BO',     'Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha No Fact',          'Nombre del proveedor','Grupo de compras']]

In [114]:
side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
side_b.rename(columns={'Statur TR No FAct':'Status','Cantidad en BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_31160\410670440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_31160\410670440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_b.rename(columns={'Statur TR No FAct':'Status','Cantidad en BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)


In [115]:
tr_final = pd.concat([side_a, side_b])

In [116]:
tr_final.to_excel("TR FINAL R3.xlsx", index=False)

In [6]:
#MARA
columnas_mara = ['Material_R3','Part_number','Material_dsc','Modelo','Familia', 'Subfamilia', 'Categoría', 'Subcatgería','Sector_dsc']
ruta_mara = ruta_repo.joinpath('MARA_R3.csv')

# Leer el archivo CSV en un DataFrame
df_mara = pd.read_csv(ruta_mara)

print('Ruta Mara: ' + '\n' + str(ruta_mara))

C:\Users\lravlic\AppData\Local\Temp\ipykernel_24072\3838735803.py:6: DtypeWarning: Columns (0,3,4,5,6,7,8,11,12,13,14,15,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mara = pd.read_csv(ruta_mara)


Ruta Mara: 
C:\Users\lravlic\Inchcape\Planificación y Compras Chile - Documentos\Bases Indicadores en CSV 2025-01\MARA_R3.csv


In [7]:
import win32com.client
import getpass
usuario = getpass.getuser()

# Initialize SAP GUI Scripting
sap_gui_auto = win32com.client.GetObject("SAPGUI")
application = sap_gui_auto.GetScriptingEngine

# Establish connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the window
session.findById("wnd[0]").maximize()

# Enter transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "me5a"
session.findById("wnd[0]").sendVKey(0)

# Set plant code
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").text = "0201"
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").caretPosition = 4
session.findById("wnd[0]").sendVKey(0)

# Set document type
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").caretPosition = 0
session.findById("wnd[0]/usr/btn%_S_BSART_%_APP_%-VALU_PUSH").press()

# Enter multiple values
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = "zsto"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "zspt"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").text = "zvor"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,3]").text = "zipl"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").text = "zatt"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").caretPosition = 4
session.findById("wnd[1]").sendVKey(8)

# Set status
session.findById("wnd[0]/usr/ctxtS_STATU-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_STATU-LOW").caretPosition = 0
session.findById("wnd[0]/usr/btn%_S_STATU_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = "a"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "n"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").caretPosition = 1
session.findById("wnd[1]").sendVKey(8)

# Execute
session.findById("wnd[0]").sendVKey(8)

# Export to Excel
session.findById("wnd[0]/tbar[1]/btn[33]").press()
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(38, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").firstVisibleRow = 30
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "38"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "ME5A_R3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 8
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the session
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)



In [8]:
me5a_df_r3 = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/ME5A_R3.XLSX")

In [9]:
me5a_df_r3 = me5a_df_r3.merge(cod_actual_df, left_on='Material', right_on='Nro_pieza_fabricante_1',how='left')

In [10]:
me5a_df_r3['Cod_Actual_1'].fillna(me5a_df_r3['Material'], inplace=True)
me5a_df_r3.drop(columns= 'Nro_pieza_fabricante_1', inplace=True)
me5a_df_r3.rename(columns={'Cod_Actual_1':'Ult. Eslabon'}, inplace=True)

In [11]:
me5a_df_r3 = me5a_df_r3[['Solicitud de pedido', 'Clase documento', 'Fecha de solicitud',
       'Pos.solicitud pedido', 'Material','Ult. Eslabon',
       'Texto breve', 'Cantidad solicitada', 'Unidad de medida',
       'Nombre del proveedor', 'Indicador de borrado', 'Status tratamiento',
       'Centro', 'Status tratamiento solicitud pedido', 'Fecha de entrega',
       'Grupo de compras', 'Solicitante', 'Proveedor deseado',
       'Proveedor fijo', 'Reg.info de compras', 'Creado por',
       'Fecha de pedido', 'Nombre del proveedor deseado', 'Pedido',
       'Posición de pedido', 'Proveedor', 'Moneda', 'Precio de valoración',
       'Valor total', 'Petición de oferta',
       'Fecha Petición de oferta', 'Texto bloqueo', 'Cantidad confirmada','Urgencia necesidad'
       ]]

In [12]:
me5a_df_r3.to_excel(destino_tubo + "ME5A R3.xlsx", index=False)

In [13]:
import win32com.client
import getpass
usuario = getpass.getuser()

# Initialize SAP GUI Scripting
sap_gui_auto = win32com.client.GetObject("SAPGUI")
application = sap_gui_auto.GetScriptingEngine

# Establish connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the window
session.findById("wnd[0]").maximize()

# Enter transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "MB52"
session.findById("wnd[0]").sendVKey(0)

# Open value help for plant field and fill in the values
session.findById("wnd[0]/usr/btn%_WERKS_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "0270"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").text = "0501"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").caretPosition = 4
session.findById("wnd[1]").sendVKey(8)

# Set the variant and execute
session.findById("wnd[0]/usr/ctxtP_VARI").text = "/LRAVLIC"
session.findById("wnd[0]/usr/ctxtP_VARI").setFocus()
session.findById("wnd[0]/usr/ctxtP_VARI").caretPosition = 8
session.findById("wnd[0]").sendVKey(8)

# Export the data
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "STOCK_R3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)



In [14]:
stock_r3 = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/STOCK_R3.XLSX")

In [15]:
stock_r3 = stock_r3.merge(cod_actual_df, left_on = 'Material', right_on='Nro_pieza_fabricante_1', how = 'left')

In [16]:
stock_r3.drop(columns='Nro_pieza_fabricante_1', inplace=True)
stock_r3.rename(columns={'Cod_Actual_1':'Ult. Eslabon'}, inplace=True)

In [17]:
stock_r3['Ult. Eslabon'].fillna(stock_r3['Material'], inplace=True)

In [18]:
stock_r3 = stock_r3.merge(df_mara, left_on='Ult. Eslabon', right_on='Material_R3')

In [19]:
stock_r3.to_excel(destino_tubo + "Stock R3.xlsx")